<a href="https://colab.research.google.com/github/Vitaly-lv/DS/blob/main/HW_2_NN_LeontevVV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import datasets
from torch.utils.data import DataLoader

Обучить полносвязную модель на MNIST

In [ ]:
dataset = datasets.MNIST('.', download=True)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw



In [ ]:
class LinearModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_p=0.1):
        super().__init__()
        self.linear1 = nn.Linear(input_dim, hidden_dim)
        self.do1 = nn.Dropout(dropout_p)
        self.activation = nn.ReLU()
        self.do2 = nn.Dropout(dropout_p)
        self.linear2 = nn.Linear(hidden_dim, output_dim)

        

    def forward(self, x):
        x = self.linear1(x)
        x = self.do1(x)
        x = self.activation(x)
        x = self.do2(x)
        x = self.linear2(x)

        return x

In [ ]:
def collate_fn(data: list):
  # data = [(pic, target)...]
  pics = []
  target = []
  for item in data:
    pics.append(numpy.array(item[0]))
    target.append(item[1])
  pics = torch.from_numpy(numpy.array(pics)).float() / 255 # B x W x H
  target = torch.from_numpy(numpy.array(target))

  return {
      'data': pics.view(pics.size(0), -1), # B x 7xx преобразуем матрицу 28х28 в вектор 7хх
      'target': target,
      }


In [ ]:
# hyperparams
inp_dim = 28 * 28
hidden = 256
out_dim = 10
device_id = -1
device = 'cpu' if device_id == -1 else f'cuda:{device_id}' # 'cuda:1' id GPU
n_epochs = 10
batch_size = 128

In [ ]:
model = LinearModel(inp_dim, hidden, out_dim).to(device)
model.train() 
optim = torch.optim.Adam(model.parameters())
loss_func = nn.CrossEntropyLoss()


In [ ]:
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset, 
                            batch_size, 
                            shuffle=True, 
                            collate_fn=collate_fn,
                            drop_last = True,
                            )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = model(batch['data'].to(device))
        loss = loss_func(predict, batch['target'].to(device))
        loss.backward()
        optim.step()
        if i % 200 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
    #save every epoch
    torch.save(model.state_dict(), f'./chkpt_cv1_{epoch}.pth')


epoch: 0, step: 0, loss: 2.3011045455932617
epoch: 0, step: 200, loss: 0.3103862702846527
epoch: 0, step: 400, loss: 0.2575129568576813
epoch: 1, step: 0, loss: 0.16210705041885376
epoch: 1, step: 200, loss: 0.1804131120443344
epoch: 1, step: 400, loss: 0.16235847771167755
epoch: 2, step: 0, loss: 0.1324893832206726
epoch: 2, step: 200, loss: 0.13524006307125092
epoch: 2, step: 400, loss: 0.1540209949016571
epoch: 3, step: 0, loss: 0.13631024956703186
epoch: 3, step: 200, loss: 0.07950906455516815
epoch: 3, step: 400, loss: 0.028262928128242493
epoch: 4, step: 0, loss: 0.07249218225479126
epoch: 4, step: 200, loss: 0.05997050926089287
epoch: 4, step: 400, loss: 0.05214628204703331
epoch: 5, step: 0, loss: 0.1005716472864151
epoch: 5, step: 200, loss: 0.06588885188102722
epoch: 5, step: 400, loss: 0.054934561252593994
epoch: 6, step: 0, loss: 0.06914285570383072
epoch: 6, step: 200, loss: 0.06908463686704636
epoch: 6, step: 400, loss: 0.031884461641311646
epoch: 7, step: 0, loss: 0.0250

Обучить глубокую сверточную сеть на MNIST

In [ ]:
class ConvNet(nn.Module): 
     def __init__(self): 
         super(ConvNet, self).__init__() 
         self.layer1 = nn.Sequential( nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=2), 
            nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2)) 
         self.layer2 = nn.Sequential( nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2), 
            nn.ReLU(), nn.MaxPool2d(kernel_size=2, stride=2)) 
         self.drop_out = nn.Dropout() 
         self.fc1 = nn.Linear(7 * 7 * 64, 1000) 
         self.fc2 = nn.Linear(1000, 10)

     def forward(self, x): 
        x = self.layer1(x) 
        x = self.layer2(x) 
        x = x.reshape(x.size(0), -1) 
        x = self.drop_out(x) 
        x = self.fc1(x) 
        x = self.fc2(x) 
        return x   

In [ ]:
# hyperparams
input_ch = 1
hidden_ch = 128
out_dim = 10
device_id = 0
device = 'cpu' if device_id == -1 else f'cuda:{device_id}'
n_epochs = 10
batch_size = 128

In [ ]:
def collate_fn_conv(data: list):
  pics = []
  target = []
  for item in data:
    pics.append(numpy.array(item[0]))
    target.append(item[1])
  pics = torch.from_numpy(numpy.array(pics)).float() / 255 # B x W x H
  target = torch.from_numpy(numpy.array(target))

  return {
      'data': pics.unsqueeze(1),
      'target': target.long(),
      }

In [ ]:
model_conv = ConvNet().to(device)
optim = torch.optim.Adam(model_conv.parameters())
loss_func = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(n_epochs):
    dataloader = DataLoader(dataset, 
                          batch_size, 
                          shuffle=True, 
                          collate_fn=collate_fn_conv,
                          drop_last = True,
                          )
    for i, batch in enumerate(dataloader):
        optim.zero_grad()

        predict = model_conv(batch['data'].to(device))
        loss = loss_func(predict, batch['target'].to(device))
        loss.backward()
        optim.step()
        if i % 200 == 0:
            print(f'epoch: {epoch}, step: {i}, loss: {loss.item()}')
    torch.save(model_conv.state_dict(), f'./chkpt_cv1_conv_{epoch}.pth')


epoch: 0, step: 0, loss: 2.31734561920166
epoch: 0, step: 200, loss: 0.06370407342910767
epoch: 0, step: 400, loss: 0.02622584067285061
epoch: 1, step: 0, loss: 0.07232273370027542
epoch: 1, step: 200, loss: 0.012452211230993271
epoch: 1, step: 400, loss: 0.1286712884902954
epoch: 2, step: 0, loss: 0.08490700274705887
epoch: 2, step: 200, loss: 0.039054449647665024
epoch: 2, step: 400, loss: 0.11704245954751968
epoch: 3, step: 0, loss: 0.0025942837819457054
epoch: 3, step: 200, loss: 0.05003463104367256
epoch: 3, step: 400, loss: 0.06076792627573013
epoch: 4, step: 0, loss: 0.01534589659422636
epoch: 4, step: 200, loss: 0.042101986706256866
epoch: 4, step: 400, loss: 0.11295168101787567
epoch: 5, step: 0, loss: 0.048770587891340256
epoch: 5, step: 200, loss: 0.022090956568717957
epoch: 5, step: 400, loss: 0.06442157924175262
epoch: 6, step: 0, loss: 0.004165331833064556
epoch: 6, step: 200, loss: 0.0016068244585767388
epoch: 6, step: 400, loss: 0.02454315312206745
epoch: 7, step: 0, lo